# <a id='toc1_'></a>[__Коллаборативная фильтрация в задаче подбора контента с наивысшим предсказанным рейтингом__](#toc0_)

**Содержание**<a id='toc0_'></a>    
- [__Коллаборативная фильтрация в задаче подбора контента с наивысшим предсказанным рейтингом__](#toc1_)    
  - [__Импорты и настройки__](#toc1_1_)    
  - [__Обзор данных__](#toc1_2_)    
  - [__Кодирование фильмов и зрителей__](#toc1_3_)    
  - [__Матрица рейтингов__](#toc1_4_)    
  - [__Похожесть между зрителями__](#toc1_5_)    
  - [__Рекомендация фильмов с наивысшим предсказанным рейтингом__](#toc1_6_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

***
## <a id='toc1_1_'></a>[__Импорты и настройки__](#toc0_)

In [1]:
# стандартная библиотека
import warnings
warnings.filterwarnings('ignore')

In [2]:
# сторонние библиотеки
from scipy.sparse import coo_matrix, csr_matrix
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import LabelEncoder
import numpy as np
import pandas as pd

***
## <a id='toc1_2_'></a>[__Обзор данных__](#toc0_)

In [ ]:
# data: https://disk.yandex.com/d/UNLSl8H9Lc0aPw

In [3]:
# таблица оценок, выставленных пользователями фильмам
rates = pd.read_csv('data/user_ratedmovies.dat', sep='\t')
rates.sample(5)  # случайная выборка из 5 записей (разметок)

,userID,movieID,rating,date_day,date_month,date_year,date_hour,date_minute,date_second
770268,64540,6724,3.5,4,1,2004,4,37,28
607119,49582,5151,2.0,24,7,2007,20,2,31
308226,25248,368,2.0,16,3,2006,22,16,23
250358,20812,2706,4.0,29,9,2002,8,22,34
325198,26617,435,0.5,16,2,2007,19,59,30


In [4]:
# таблица атрибутов фильмов 
movies = pd.read_csv('data/movies.dat', sep='\t', encoding='iso-8859-1')
movies.sample(5)  # случайная выборка из 5 записей (фильмов)

,id,title,imdbID,spanishTitle,imdbPictureURL,year,rtID,rtAllCriticsRating,rtAllCriticsNumReviews,rtAllCriticsNumFresh,...,rtAllCriticsScore,rtTopCriticsRating,rtTopCriticsNumReviews,rtTopCriticsNumFresh,rtTopCriticsNumRotten,rtTopCriticsScore,rtAudienceRating,rtAudienceNumRatings,rtAudienceScore,rtPictureURL
9395,50800,The Messengers,425430,The Messengers,http://ia.media-imdb.com/images/M/MV5BMTMxMjMz...,2007,the_messengers,3.7,83,10,...,12,3.6,19,1,18,5,3.1,129304,48,http://content6.flixster.com/movie/10/89/27/10...
1333,1472,City of Industry,118859,Ajuste de cuentas,http://ia.media-imdb.com/images/M/MV5BMjA2MjI0...,1997,city_of_industry,5.2,13,6,...,46,4.7,6,1,5,16,2.9,791,31,http://content8.flixster.com/movie/27/54/27549...
6168,6543,The Holy Land,283387,The Holy Land,http://ia.media-imdb.com/images/M/MV5BMjE5NzMw...,2001,holy_land,5.8,54,28,...,51,5.8,19,10,9,52,3.6,173,68,http://content8.flixster.com/movie/10/88/43/10...
9,10,GoldenEye,113189,GoldenEye,http://ia.media-imdb.com/images/M/MV5BNTE1OTEx...,1995,goldeneye,6.8,41,33,...,80,6.2,11,7,4,63,3.4,28260,78,http://content9.flixster.com/movie/26/66/26669...
253,264,L'enfer,109731,El infierno (L'enfer),http://ia.media-imdb.com/images/M/MV5BMTc2OTI1...,1994,1059177-lenfer,7.9,8,8,...,100,0,1,1,0,100,3.5,421,67,http://content8.flixster.com/movie/10/88/73/10...


***
## <a id='toc1_3_'></a>[__Кодирование фильмов и зрителей__](#toc0_)

In [5]:
# оставим фильмы, которые имеют хотя бы одну зрительскую оценку 
movies = movies[movies.id.isin(rates.movieID)]

In [6]:
# всего таких фильмов 10109
movies.shape

(10109, 21)

Закодируем зрителей в таблице оценок и согласованно, т.е. общим кодировщиком, — фильмы в таблице зрителей и таблице фильмов: 

[__sklearn.preprocessing.LabelEncoder__](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelEncoder.html#sklearn-preprocessing-labelencoder)

> Encode target labels with value between 0 and n_classes-1.<br>
This transformer should be used to encode target values.

In [7]:
user_enc, movies_enc = LabelEncoder(), LabelEncoder()

In [8]:
rates.userID = user_enc.fit_transform(rates.userID)
rates.movieID = movies_enc.fit_transform(rates.movieID)
movies.id = movies_enc.transform(movies.id)

In [9]:
rates

,userID,movieID,rating,date_day,date_month,date_year,date_hour,date_minute,date_second
0,0,2,1.0,29,10,2006,23,17,16
1,0,31,4.5,29,10,2006,23,23,44
2,0,105,4.0,29,10,2006,23,30,8
3,0,151,2.0,29,10,2006,23,16,52
4,0,154,4.0,29,10,2006,23,29,30
...,...,...,...,...,...,...,...,...,...
855593,2112,9023,4.0,3,12,2007,3,5,38
855594,2112,9111,4.0,3,12,2007,2,56,44
855595,2112,9211,4.5,3,12,2007,2,53,46
855596,2112,9900,5.0,10,10,2008,9,56,5


***
## <a id='toc1_4_'></a>[__Матрица рейтингов__](#toc0_)

Возьмем из таблицы рейтингов `rates` только нужную информацию — метки пользователей, метки фильмов и рейтинги фильмов, выставленные пользователями фильмам — и сделаем из нее разреженную таблицу, в которой на пересечении строк-пользователей и столбцов-фильмов стоят соответствующие рейтинги. Если пользователь не оценивал фильм, то его оценка 0.

[__scipy.sparse.coo_matrix__](https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.coo_matrix.html#scipy-sparse-coo-matrix)

> A sparse matrix in COOrdinate format.

In [10]:
R = coo_matrix((rates.rating, (rates.userID, rates.movieID)))
R  # 2113 зрителей x 10109 фильмов

<2113x10109 sparse matrix of type '<class 'numpy.float64'>'
	with 855598 stored elements in COOrdinate format>

In [11]:
type(R)

scipy.sparse._coo.coo_matrix

Конвертируем разреженную матрицу `R` в сжатый формат:

[__scipy.sparse.csr_matrix__](https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.csr_matrix.html#scipy-sparse-csr-matrix)

> Convert matrix to Compressed Sparse Row format.

In [12]:
R = R.tocsr()
type(R)

scipy.sparse._csr.csr_matrix

Вектор оценок пользователя можно получить обращением по индексу. Например, пользователь, закодированный нулем, выставил оценки 55 фильмам: 

In [13]:
R[0]

<1x10109 sparse matrix of type '<class 'numpy.float64'>'
	with 55 stored elements in Compressed Sparse Row format>

Проверим это:

In [14]:
rates.query('userID == 0').rating.size

55

> with 55 stored elements

***
## <a id='toc1_5_'></a>[__Похожесть между зрителями__](#toc0_)

[__sklearn.metrics.pairwise.cosine_similarity__](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise.cosine_similarity.html#sklearn-metrics-pairwise-cosine-similarity)

> Compute cosine similarity between samples in X and Y.<br>
Cosine similarity, or the cosine kernel, computes similarity as the normalized dot product of X and Y:<br>
$\huge K(X, Y) = \frac{<X, Y>}{||X||\cdot||Y||}$

Это косинус угла между векторами, выраженный через их скалярное произведение. Соответственно, чем более похожи векторы, т.е. чем меньше угол между ними, тем ближе их косинусная похожесть к единице.

In [15]:
def cosineSimilarity(u, v, thres=3):
    """
    Похожесть между парой зрителей на фильмах, размеченных обоими.
    Если число таких фильмов меньше thres, похожесть считать нулевой.
    """
    # произведение отлично от нуля, если оба множителя ненулевые
    indices = u.multiply(v).nonzero()[1]
    if indices.size < thres:
        return 0
    return cosine_similarity(u[:,indices], v[:,indices]).item()

In [16]:
cosineSimilarity(R[146], R[239])  # пример

0.9228878934866824

In [17]:
def similar_users(index, R, n_neigbors):
    """
    Функция возвращает кортеж:
    (0) Индексы n_neigbors зрителей, наиболее косинусно похожих на зрителя
        с индексом index в матрице рейтингов R, включая его самого.
    (1) Косинусные схожести, соответствующие зрителям-соседям из (0).
    """
    # попарные косинусные похожести пользователя index 
    # со всеми пользователями из R, включая его самого
    similar = np.array([cosineSimilarity(R[index], user) for user in R])
    
    # первые n_neigbors индексов, сортирующие по убыванию косинусной 
    # похожести - от более похожих на пользователя index к менее
    indices = np.argsort(-similar)[:n_neigbors]
    
    return indices, similar[indices]

In [18]:
similar_users(42, R, n_neigbors=10)[0]  # пример

array([  42,  281,  633,  724,  815,    2,  620,  650, 1692, 1506],
      dtype=int64)

Первым в списке наиболее похожих, разумеется, стоит сам пользователь — его косинусная похожесть на самого себя ровно единица.

***
## <a id='toc1_6_'></a>[__Рекомендация фильмов с наивысшим предсказанным рейтингом__](#toc0_)

Составим рекомендацию из 5 фильмов для пользователя с индексом 20 по 30 наиболее похожим на него с учетом следующих условий:

* сам пользователь учитываться не должен;
* среди рекомендуемой пятерки не должно быть ранее просмотренных фильмов.

Для выполнения первого условия нужно взять 31 наиболее косинусно похожих пользователей и отбросить первого — самого себя:

In [19]:
exclude_self = lambda arr: arr[1:]
indices, similar = map(exclude_self, similar_users(20, R, n_neigbors=31))

Извлечем из матрицы рейтингов оценки 30 ближайших пользователей:

In [20]:
neighbors_rates = R[indices].toarray()
neighbors_rates.shape

(30, 10109)

Для получения вектора предсказания нужно умножить оценки ближайших пользователей на соответствующие косинусные похожести (веса) и нормировать суммой весов. Иными словами, предсказание рейтинга, который пользователь, вероятно, поставил бы некоторому фильму, есть средневзвешенная оценка этого фильма некоторого количества наиболее похожих на него пользователей:

In [21]:
predictions = neighbors_rates.T @ similar / similar.sum()
predictions.shape

(10109,)

Вытащим из матрицы рейтингов фильмы, которые разметил 20-ый пользователь, и обнулим в векторе предсказаний соответствующие оценки — оценки фильмам, которые 20-ый пользователь уже видел и оценил сам: 

In [22]:
true_rates = R[20].toarray()

# зануляем предсказания для фильмов, которые пользователь 
# уже видел - эти фильмы не участвуют в отборе
predictions[np.nonzero(true_rates)[1]] = 0

Найдем первые 5 индексов, которые сортируют вектор предсказаний по убыванию оценки — это и есть идентификаторы пяти фильмов, которые и составляют нашу рекомендацию:  

In [23]:
top = 5
np.argsort(-predictions)[:top]

array([2614,  306,  343, 5573, 6720], dtype=int64)

***